In [1]:
import sys
sys.path.append('..')

In [59]:
import src
from shapely import wkt
import geopandas as gpd
import pandas as pd
from tqdm import tqdm

# Polígonos del RENABAP
# https://datos.gob.ar/dataset/habitat-registro-nacional-barrios-populares
renabap = gpd.read_file('https://archivo.habitat.gob.ar/dataset/ssisu/renabap-datos-barrios-geojson')

In [57]:
mask = wkt.loads(
    'POLYGON((-58.9894 -34.2766, -57.908 -34.2766, -57.908 -34.9895, -58.9894 -34.9895, -58.9894 -34.2766))'
)

In [ ]:
renabap = renabap[renabap.intersects(mask)]

In [20]:
renabap_buffered = renabap.copy()
renabap_buffered['geometry'] = renabap_buffered.to_crs(3857).buffer(500).to_crs(4326)

renabap_buffered = gpd.GeoDataFrame(
    [renabap_buffered.union_all()], columns = ['geometry']
).reset_index(drop=True).explode()

In [48]:
distpoints = 75
points = src.utils.create_point_grid(renabap_buffered.geometry.iloc[0], distpoints)


points = gpd.GeoDataFrame(points, columns=["geometry"], crs=4326)

In [49]:
panos = src.sv.get_panos(points)

100%|██████████████████████████████████████████████████████████| 1283/1283 [03:23<00:00,  6.30it/s]


In [50]:
panos.columns = ['id','lat','lon','date']

In [51]:
panos.to_csv(f'panos_{distpoints}mdist.csv')

In [52]:
panos.id.nunique()

1634

In [61]:
def loop_polys_get_panos(gdf, buffer_dist, dist_points, mask=None):
    output = []
    gdf_buffered = gdf.copy()
    if mask:
        gdf_buffered = gdf_buffered[gdf_buffered.intersects(mask)]
    gdf_buffered['geometry'] = gdf_buffered.to_crs(3857).buffer(buffer_dist).to_crs(4326)
    gdf_buffered = gpd.GeoDataFrame(
        [gdf_buffered.union_all()], columns = ['geometry']
    ).reset_index(drop=True).explode()
    for _, row in tqdm(gdf_buffered.iterrows(), total=len(gdf_buffered)):
        geometry = row['geometry']
        points = src.utils.create_point_grid(geometry, dist_points)
        points = gpd.GeoDataFrame(points, columns=["geometry"], crs=4326)
        panos = src.sv.get_panos(points)
        panos.columns = ['id','lat','lon','date']
        output.append(panos)

    return pd.concat(output).reset_index(drop=True)

In [62]:
panos = loop_polys_get_panos(renabap, buffer_dist=500, dist_points=50, mask=mask)

  0%|▏                                                           | 1/255 [01:19<5:34:34, 79.03s/it]

KeyboardInterrupt

